# Python + Pandas is love

![Logo Pandas](pics/pandas_logo.png)

L'idéal pour une rapport de données, la validation d'une intuition, vite fait, sur un coin de table, en mode oneshot / quick&dirty ? ~~Excel~~ [Pandas](https://pandas.pydata.org/) !

Nous allons suivre la génération d'un rapport à partir d'un _petit_ dump de données, ce qui va nous permettre de faire une visite guidée des fonctionnalités de pandas. Ce ne sera pas exhaustif, mais devrait vous permettre d'y penser la prochaine fois. 

[Version page web](https://nbviewer.jupyter.org/github/flo-dhalluin/python-pandas-talk/blob/master/pandas-talk-run.ipynb)

In [ ]:
%matplotlib inline

## La base


Sortons les outils : 

In [ ]:
import pandas as pd 
import numpy as np

### La base : Serie

Une série, est une séquence de valeurs, de type homogène (toutes du même type : String, entiers, flotants, dates ... ), __ avec un index __

In [ ]:
pd.Series([6,5,4,3,2,1], index=10 * np.arange(6)) # index par default : 0, 1 .. 


### La base : Dataframe

Un Dataframe, c'est un tableau : 
- un ensemble de Series, nommées ( les colonnes)
- qui partagent un index


In [ ]:
data = pd.DataFrame({"time": pd.date_range("2018-01-01", periods=120, freq="2H"),
                     "value": np.random.random(120),                            
                     "category": pd.Categorical(list("abcdef" * 20))}) 
data.head()

### La base : Import/Export

L'idée étant évidemment d'importer nos données ici, et pas de les générer. Pandas supporte à peu près tout les fichiers formats de données courants :
- csv 
- excel 
- hdf5 
- json/msgpack/parquet

_mais aussi_  on peut créer un dataframe directement depuis une requête SQL, ou même Big Query.

Nous allons ici utiliser la lingua franca de la data ( rappel du contexte : on est sale, on est rapide, on est pas sexys ...) : le csv. 

In [ ]:
events = pd.read_csv("events.csv", 
                     delimiter=";", 
                     index_col=0,
                     parse_dates=True) # auto magie. 
events.sort_index(inplace=True)
events.head(5)

Une petite vérif rapido :

In [ ]:
events.describe()

Notons bien que : il n'y a que 4 valeurs possibles pour doc_type et beaucoup de "non-valeurs" pour doc_type donc : 
on nettoye tout ça. 

In [ ]:
events.type = events.type.astype("category")
events.doc_type = events.doc_type.fillna("UNKNOWN").astype("category")


### Le problème
Nous voudrions compiler un peu des stats sur le nombre de soumission, par type de document ... ( combien de soumission par dossier / type de document, le timing .. )


### Requêtes simples : par index

<font color="red"> Bon gros warning :   l'opérateur `[]` est contre-intuitif. </font>


Pour retrouver des lignes par index :  c'est `.iloc` (index numérique ) ou `.loc` (index déclaré )

In [ ]:
events.iloc[241]  # par index de la ligne

On peut évidemment utiliser la syntaxe slice de python : (rappel : l'index est le timestamp)

In [ ]:
# Tous les évenements du 10 janvier
events.loc['2017-01-10':'2017-01-11']

En fait Il y a 2 cas d'usages pour `[]` 

selection d'une ou plusieurs colonnes :



In [ ]:
events[["doc_type", "status"]].head()

Ou pour __filtrer__ : avec une série de booléens

In [ ]:
import datetime
# Les events concernant des CNI, le 10 janvier
(events[ (events.doc_type == "CNI") 
       & (events.index.date == datetime.date(2017, 1, 10)) ]
       .head(5))


Ou encore `query()` qui est parfois plus lisible, et plus puissante :

In [ ]:
# un peu plus lisible : les CNI avant le 10 janvier, en errer ( status : False)
events.query('doc_type=="CNI" & index < "20170110" & ~status').head()

## GroupBy

> Tableau croisé dynamique, c'est mieux si on en reste là.

Passons dans le vif du sujet, et moulinons un peu nos datas...

In [ ]:
# comptons les soumissions de documents par type/status. 

(events.groupby(('doc_type','status'))
       .count()) 


![groupby](./pics/scan_groupby.jpeg)

In [ ]:
events["timestamp"] = events.index # je crée une colonne ( l'index est perdu dans un groupby )

# Comme on créé une nouvelle colonne
events["submission_idx"] = (events.groupby(("client_uuid", "doc_type"))
                                  ["timestamp"]  # uniquement sur la colonne "timestamp" 
                                  .transform(np.argsort))
events.head()

Cette fois ci, je veux calculer le temps passé par utilisateur _depuis le premier évenement lié à cet utilisateur_

In [ ]:
events['time_since_first'] = (events.groupby('client_uuid')
                              ["timestamp"]
                              .transform(lambda ts: ts-ts[0])) # C'est là !

events.sort_values(by=["client_uuid", "timestamp"]).head()

statistiques du nombre de soumissions par type de doc (et après on arrête... )

In [ ]:
(events.groupby(("doc_type", "submission_idx"))
                .time_since_first
                .aggregate({"counts": "count",
                            "mean_time": lambda g: g.mean()})) # "mean" string shortcut does not work on timedeltas

### Pivot 

En fait, il y a un autre moyen de faire le dernier calcul, comme s'intéresse uniquement au nombre total de soumissions par type de documents : `.pivot`

In [ ]:
submission_counts = pd.pivot_table(events, 
                   index="client_uuid", # une ligne par client_uuid. ok
                   columns="doc_type", # une colonne par type de doc
                   values="timestamp", # du coup : on a un timestamp par soumission
                   aggfunc="count")    # qu'on compte.

submission_counts.head()

In [ ]:
submission_counts.describe()

### Des graphiques ! 

In [ ]:
# temps passé par dossier

(events.groupby('client_uuid') # Tu vois ce que je veux dire ? 
     .time_since_first # juste le temps depuis la première action
     .max().map(lambda x: x.total_seconds()) # timedelta -> "float" ( seconds )
     .hist(bins=50)) # KABOOM !


### Et maintenant ? 

Vous avez la base, vous pouvez plonger plus loin : 
- `.merge` et `.join` : si j'ai 2 tables, des jointures comme à la maison ! 
- les timeseries, et tout ce qu'on peut faire avec ( resampling, rolling windows, conversions .. )

Et une fois que vous avez bien tout nettoyé, évidement, on peut jouer avec scikit-learn pour faire du Machine learning, ou [bokeh](https://bokeh.pydata.org/) pour des zouli graphiques, direct depuis pandas.

# Merci ! 

- [sources](https://github.com/flo-dhalluin/python-pandas-talk)
- fait avec [jupyter / notebook](https://jupyter.org/)
- Le saviez vous ? vous pouvez exporter un notebook en slides/reveal.js
